# import packages

In [ ]:
import pathlib
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from scipy.signal import convolve
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d import Axes3D
from numpy.fft import fftn, fftshift
from tqdm.auto import tqdm
import os

import dask
import dask.array as da
from dask.diagnostics import ProgressBar
import flox
import flox.xarray
from flox.xarray import xarray_reduce
import xrft

%matplotlib widget

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:43027")
client

# import local files

In [ ]:
from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz, rotation_matrix, gaussian_kernel
from meshgrids import generate_density_grid, convert_grid_qspace, plot_3D_grid, generate_electron_grid_npys, load_npy_files_to_dask
from detector import make_detector, rotate_about_normal, rotate_about_horizontal, rotate_about_vertical, intersect_detector

# Generate and plot real-space voxel map for xyz file

In [ ]:
# Define base path
basePath = pathlib.Path('/nsls2/users/alevin/repos/giwaxs_forward_sim')
xyzPath = basePath.joinpath('test_xyz_files/graphite_small.xyz')
npySavePath = pathlib.Path('/nsls2/users/alevin/misc_data/density_grid_segments/graphite_medium')

sigma = 0.2
voxel_size = 0.05
min_ax_size = 1024
segments = 8  # segments along x

x_axis, y_axis, z_axis, grid_vox_x, grid_vox_y, grid_vox_z = generate_electron_grid_npys(xyzPath, 
                                                                                         voxel_size, 
                                                                                         segments,
                                                                                         npySavePath,
                                                                                         sigma,
                                                                                         min_ax_size=min_ax_size)

# Below loads the numpy array stacks into a dask array
# I've so far been unable to fit this all into the separate python script without 
# running into strange moduleimport errors... but this should work!
def load_array_from_npy_stack(npy_paths):
    """"""
    arrs = []
    for npy_path in npy_paths:
        arr = np.load(npy_path)
        arrs.append(arr)

    return np.concatenate(arrs, axis=1)   

npy_paths = sorted(npySavePath.glob('*.npy'))
density_grid = dask.delayed(load_array_from_npy_stack)(npy_paths)
density_grid = dask.array.from_delayed(density_grid, shape=(grid_vox_y, grid_vox_x, grid_vox_z), dtype=float)
density_grid = density_grid.rechunk((grid_vox_y, int(grid_vox_x/8), grid_vox_z))

density_grid = density_grid.persist()
density_grid

In [ ]:
# # Populate grid segments:
# # Sort coordinates & symbols by the x coordinate value (by entering as the last column in np.lexsort)
# ind = np.lexsort((coords[:,2], coords[:,1], coords[:,0]))  # return sorted indices values (first value)
# # ind = np.lexsort((coords[:,2], coords[:,0], coords[:,1]))  # return sorted indices values (first value)
# # ind = np.lexsort((coords[:,0], coords[:,1], coords[:,2]))  # return sorted indices values (first value)
# symbols = symbols[ind]
# coords = coords[ind]

In [ ]:
# Create a new array to store the shifted values
shifted_cuts = np.zeros_like(coord_idx_cuts, dtype=float)

# Compute the shifted values
shifted_cuts[0] = coord_idx_cuts[0] / 2  # First element is the average of itself and zero
for i in range(1, len(coord_idx_cuts)):
    shifted_cuts[i] = (coord_idx_cuts[i] + coord_idx_cuts[i - 1]) / 2
    
shifted_cuts = shifted_cuts.astype(int)

In [ ]:
taken_coords = coords[:,axis][coord_idx_cuts]
# Create a new array to store the shifted values
empty_coords = np.zeros_like(taken_coords, dtype=float)
# Compute the shifted values
empty_coords[0] = taken_coords[0] / 2  # First element is the average of itself and zero
for i in range(1, len(taken_coords)):
    empty_coords[i] = (taken_coords[i] + taken_coords[i - 1]) / 2
empty_coords

In [ ]:
coords[:,axis][shifted_cuts]

In [ ]:
diff_threshold = voxel_size + (3 * sigma)  # set diff threshold by voxel size + (3*sigma) 
diff_axis = np.diff(coords, axis=axis)[:, 0]  # get difference values array
diff_axis = np.append(diff_axis, 0)  # add extra zero at the end to make same shape as coords to use as mask
coord_idx_cuts = np.nonzero(diff_axis>diff_threshold)[0] + 1 # find indices with difference value from previous greater than diff threshold 

# grid_vox_axis = (coords[:,axis][coord_idx_cuts] / voxel_size).astype(int)  # convert to number of voxels, these are allowed positions to segment
grid_vox_axis = (coords[:,axis][shifted_cuts] / voxel_size).astype(int)  # convert to number of voxels, these are allowed positions to segment

print('num idx vox')
for num, coord_idx, empty_idx, vox in zip(np.arange(len(coord_idx_cuts)), coord_idx_cuts, shifted_cuts, grid_vox_axis):
    print(num, coord_idx, empty_idx, vox)

In [ ]:
vox_break_targs = np.arange(grid_vox_segment, min_ax_size, grid_vox_segment)

vox_breaks = np.array([0, min_ax_size])
for vox_break_targ in vox_break_targs:
    if vox_break_targ > grid_vox_axis[-1]:
        vox_breaks = np.append(vox_breaks, vox_break_targ)
    else:
        vox_break_diffs = np.abs(grid_vox_axis - vox_break_targ)
        vox_break_shifted = grid_vox_axis[vox_break_diffs==vox_break_diffs.min()][0]
        vox_breaks = np.append(vox_breaks, vox_break_shifted)
        
vox_breaks.sort()

vox_axis_mins = vox_breaks[:-1]
vox_axis_maxs = vox_breaks[1:]
vox_axis_slices = [(vox_axis_min,vox_axis_max) for vox_axis_min,vox_axis_max in zip(vox_axis_mins,vox_axis_maxs)]
vox_axis_slices

In [ ]:
# Find all voxels along axis where there is enough space between point smeared with gaussian
diff_threshold = voxel_size + (5 * sigma)  # set diff threshold by voxel size + (3*sigma) 
diff_axis = np.diff(coords, axis=axis)[:, 0]  # get difference values array
diff_axis = np.append(diff_axis, 0)  # add extra zero at the end to make same shape as coords to use as mask
coord_idx_cuts = np.nonzero(diff_axis>diff_threshold)[0] + 1  # find indices with difference value from previous greater than diff threshold 
grid_vox_axis = (coords[:,axis][coord_idx_cuts] / voxel_size).astype(int)  # convert to number of voxels, these are allowed positions to segment


diff_axis

In [ ]:
## Cell for testing with segments being inputted as carefully selected list of grid_vox_x segments

# Define base path
basePath = pathlib.Path('/nsls2/users/alevin/repos/giwaxs_forward_sim')
xyz_path = basePath.joinpath('test_xyz_files/graphite_medium.xyz')
npySavePath = pathlib.Path('/nsls2/users/alevin/misc_data/density_grid_segments/',
                           'graphite_medium_testing')
npySavePath.mkdir(exist_ok=True)

sigma = 0.1
voxel_size = 0.045
min_ax_size = 512
axis_to_chunk = 'x'
segments_type = 'flexible'  # for spacing between splits in the data, vs 'fixed' for fixed integer value
# segments = 8  # if segments type if 'fixed', number of segments along axis to chunk
grid_vox_segment = 128  # if segments type is 'flexible', target number for grid_vox_x

# Extracting the atomic symbols and positions from the xyz file
coords, symbols = load_xyz(xyz_path)

# Shift coords array to origin (buffer ensures room for Gaussian smearing)
buffer = 3 * sigma # same size as guassian kernel (made later)
coords[:,0] -= np.min(coords[:,0])-buffer
coords[:,1] -= np.min(coords[:,1])-buffer
coords[:,2] -= np.min(coords[:,2])-buffer

# Axis grids
grid_size_x = int(np.ceil((np.max(coords[:,0])+buffer)/voxel_size))
grid_size_y = int(np.ceil((np.max(coords[:,1])+buffer)/voxel_size))
grid_size_z = int(np.ceil((np.max(coords[:,2])+buffer)/voxel_size))

# Calcuate number of voxel grid points, pad to nearest 2^n
grid_vox_x = 1 << (grid_size_x - 1).bit_length()
grid_vox_y = 1 << (grid_size_y - 1).bit_length()
grid_vox_z = 1 << (grid_size_z - 1).bit_length()
if grid_vox_x < min_ax_size:
    grid_vox_x = min_ax_size
if grid_vox_y < min_ax_size:
    grid_vox_y = min_ax_size
if grid_vox_z < min_ax_size:
    grid_vox_z = min_ax_size

# Create axes
x_axis = np.linspace(0, grid_vox_x*voxel_size, grid_vox_x)
y_axis = np.linspace(0, grid_vox_y*voxel_size, grid_vox_y)
z_axis = np.linspace(0, grid_vox_z*voxel_size, grid_vox_z)

# Sort coordinates & symbols by the specifed axis coordinate value (by entering as the last column in np.lexsort)
# Extract int for axis to chunk
if axis_to_chunk == 'x':
    axis = 0
    ind = np.lexsort((coords[:,2], coords[:,1], coords[:,0]))
elif axis_to_chunk == 'y':
    axis = 1
    ind = np.lexsort((coords[:,2], coords[:,0], coords[:,1]))
elif axis_to_chunk == 'z':
    axis = 2
    ind = np.lexsort((coords[:,0], coords[:,1], coords[:,2]))
symbols = symbols[ind]
coords = coords[ind]

# Find all voxels along axis where there is enough space between point smeared with gaussian
diff_threshold = voxel_size + (5 * sigma)  # set diff threshold by voxel size + (3*sigma) 
diff_axis = np.diff(coords, axis=axis)[:, 0]  # get difference values array
diff_axis = np.append(diff_axis, 0)  # add extra zero at the end to make same shape as coords to use as mask
coord_idx_cuts = np.nonzero(diff_axis>diff_threshold)[0] # + 1  # find indices with difference value from previous greater than diff threshold 
taken_coords = coords[:,axis][coord_idx_cuts]
# Create a new array to store the shifted values
empty_coords = np.zeros_like(taken_coords, dtype=float)
# Compute the shifted values
empty_coords[0] = taken_coords[0] / 2  # First element is the average of itself and zero
for i in range(1, len(taken_coords)):
    empty_coords[i] = (taken_coords[i] + taken_coords[i - 1]) / 2
grid_vox_axis = (empty_coords / voxel_size).astype(int)  # convert to number of voxels, these are allowed positions to segment

# Based on allowed voxel points above, define voxel slices along specified axis
vox_break_targs = np.arange(grid_vox_segment, min_ax_size, grid_vox_segment)  # target voxel breakpoints (not including 0 & end)

# Generate actual allowed break points
vox_breaks = np.array([0, min_ax_size])
for vox_break_targ in vox_break_targs:
    if vox_break_targ > grid_vox_axis[-1]:
        vox_breaks = np.append(vox_breaks, vox_break_targ)
    else:
        vox_break_diffs = np.abs(grid_vox_axis - vox_break_targ)
        vox_break_shifted = grid_vox_axis[vox_break_diffs==vox_break_diffs.min()][0]
        vox_breaks = np.append(vox_breaks, vox_break_shifted)
vox_breaks.sort()

# Reshape breakpoints into tuples of min/max
vox_axis_mins = vox_breaks[:-1]
vox_axis_maxs = vox_breaks[1:]
vox_axis_slices = [(vox_axis_min,vox_axis_max) for vox_axis_min,vox_axis_max in zip(vox_axis_mins,vox_axis_maxs)]
x_mins = x_axis[vox_axis_mins]
x_maxs = x_axis[vox_axis_maxs[:-1]]
x_maxs = np.append(x_maxs, x_axis[-1])

# Loop over each vox segment to populate
grid_vox_segments = np.array([])
for i, vox_axis_slice in enumerate(tqdm(vox_axis_slices, desc='Populating & saving grid segments')):
    # Get segment size
    grid_vox_axis_segment = vox_axis_slices[i][1] - vox_axis_slices[i][0]
    grid_vox_segments = np.append(grid_vox_segments, grid_vox_axis_segment)
    
    if axis_to_chunk=='x':
        # Create empty grid segment
        density_grid_segment = np.zeros((grid_vox_y, grid_vox_axis_segment, grid_vox_z))
        
        # Slice x_axis for segment to identify x maximum and minimum
        # x_axis_slice = x_axis[ i * grid_vox_axis_segment:
        #                       (i+1) * grid_vox_axis_segment ]
        x_min = x_mins[i]
        x_max = x_maxs[i]

        # Downselect pre-sorted coords & symbols segment to apply to loop
        segment_coords_mask = (coords[:,axis] >= x_min) & (coords[:,axis] < x_max)
        segment_coords = coords[segment_coords_mask]
        segment_symbols = symbols[segment_coords_mask]

        # Populate the grid 
        print(int(grid_vox_segments.sum()))
        for coord, symbol in zip(segment_coords, segment_symbols):
            grid_coord = np.round((coord / voxel_size),0).astype('int')
            density_grid_segment[ grid_coord[1], 
                                 (grid_coord[0]-(int(grid_vox_segments.sum()))),  # here I think is the error! 
                                  grid_coord[2] ] += (ptable[symbol])  
            
        # Create a Gaussian kernel
        if sigma:
            sigma_voxel = sigma/voxel_size
            kernel_size = 6 * sigma_voxel + 1  # Ensure the kernel size covers enough of the Gaussian
            gaussian_kernel_3d = gaussian_kernel(kernel_size, sigma_voxel)
            # convolve gaussian with 
            density_grid_segment = convolve(density_grid_segment, gaussian_kernel_3d, mode='same')
            print('did_convolution')
            
        npy_savename = f'grid_segment_along-x_num-{i}_shape-{grid_vox_y}-{grid_vox_axis_segment}-{grid_vox_z}.npy'
        np.save(npySavePath.joinpath(npy_savename), density_grid_segment)
            
    elif axis_to_chunk=='y':
        # density_grid_segment = np.zeros((grid_vox_axis_segment, grid_vox_x, grid_vox_z))
        print('not implemented yet')
    elif axis_to_chunk=='z':
        # density_grid_segment = np.zeros((grid_vox_y, grid_vox_x, grid_vox_axis_segment))
        print('not implemented yet')


In [ ]:
vox_axis_slices

In [ ]:
# Below loads the numpy array stacks into a dask array
# I've so far been unable to fit this all into the separate python script without 
# running into strange moduleimport errors... but this should work!
def load_array_from_npy_stack(npy_paths):
    """"""
    arrs = []
    for npy_path in npy_paths:
        arr = np.load(npy_path)
        arrs.append(arr)

    return np.concatenate(arrs, axis=1)   

npy_paths = sorted(npySavePath.glob('*.npy'))
density_grid = dask.delayed(load_array_from_npy_stack)(npy_paths)
density_grid = dask.array.from_delayed(density_grid, shape=(grid_vox_y, grid_vox_x, grid_vox_z), dtype=float)
# density_grid = density_grid.rechunk((grid_vox_y, int(grid_vox_x/8), grid_vox_z))
density_grid = density_grid.rechunk({0:-1, 1:int(grid_vox_x/8), 2:-1})

density_grid = density_grid.persist()
density_grid

In [ ]:
%%time

# put loaded numpy array into xarray dataarray
dens_grid_DA = xr.DataArray(data=density_grid,
                            dims=['y', 'x', 'z'],
                            coords={'y':y_axis,
                                    'x':x_axis,
                                    'z':z_axis})

# # Dask-ify it
# num_chunks = 8
# dens_grid_DA = dens_grid_DA.chunk({'x':int(len(dens_grid_DA.x)/num_chunks)})  # chunk along just one dimension, for slab fftw(?)
dens_grid_DA

In [ ]:
density_grid.sum(axis=2).shape

In [ ]:
summed_z_arr = density_grid.sum(axis=2).compute()
summed_z_arr.shape

In [ ]:
plt.close('all')
# smeared_z_arr = gaussian_filter(summed_z_arr, sigma=5)
plt.imshow(density_grid.sum(axis=2).compute(), origin='lower')
plt.show()

In [ ]:
# %%time
# dens_grid_DA.reduce(np.nanpercentile, q=0.1)

In [ ]:
# %%time
# lazy_clims = da.percentile(dens_grid_DA.data.ravel(), [0.1, 0.99])
# lazy_clims

In [ ]:
%%time
lazy_binned_DA = dens_grid_DA.groupby_bins('x', 128).mean().groupby_bins('y',128).mean().groupby_bins('z',128).mean()

In [ ]:
lazy_binned_DA.data.visualize()

In [ ]:
%%time
binned_DA = lazy_binned_DA.persist()

display(binned_DA)

In [ ]:
binned_DA = binned_DA.assign_coords({
            'x': ('x_bins', np.array([interval.mid for interval in binned_DA.x_bins.data])),
            'y': ('y_bins', np.array([interval.mid for interval in binned_DA.y_bins.data])),
            'z': ('z_bins', np.array([interval.mid for interval in binned_DA.z_bins.data]))
                   }).swap_dims({'x_bins':'x', 'y_bins':'y', 'z_bins':'z'})
binned_DA

In [ ]:
plt.close('all')
threshold = 99.9
num_levels = 10
cmap = 'plasma'
fig, ax = plot_3D_grid(density_grid.compute(), x_axis, y_axis, z_axis, cmap, threshold, num_levels, log=True)
# fig, ax = plot_3D_grid(binned_DA.data.compute(), binned_DA.x.data, binned_DA.y.data, binned_DA.z.data, cmap, threshold, num_levels, log=True)

plt.show()

# Generate and plot reciprocal space voxel map for xyz file

In [ ]:
def xrft_iq(DA, num_chunks):
    fft_yz = xrft.fft(DA, dim=['y','z'])  # take dft in y & z direction
    fft_yz_rechunked = fft_yz.chunk({'freq_y':int(len(DA.y))/num_chunks,'x':int(len(DA.x))})  # rechunk along y direction 
    fft_all = xrft.fft(fft_yz_rechunked, dim=['x'])  # take dft in x direction
    iq_DA = np.abs(fft_all)**2
    
    return iq_DA

In [ ]:
%%time
num_chunks = 8
iq_DA = xrft_iq(dens_grid_DA, num_chunks)  #.compute()
iq_DA

In [ ]:
iq_DA.data.visualize()

In [ ]:
%%time 
iq_DA = iq_DA.persist()
iq_DA

In [ ]:
%%time
fft_yz = xrft.fft(dens_grid_DA, dim=['y','z'])  # take dft in y & z direction
fft_yz_rechunked = fft_yz.chunk({'freq_y':int(len(dens_grid_DA.y))/8,'x':int(len(dens_grid_DA.x))})  # rechunk along y direction 
fft_all = xrft.fft(fft_yz_rechunked, dim=['x'])  # take dft in x direction
# with ProgressBar():

# fft_all = fft_all.persist()

fft_all

In [ ]:
fft_yz.data.visualize()

In [ ]:
fft_all.data.visualize()

In [ ]:
fft_all = fft_all.compute()
fft_all.data.visualize()

In [ ]:
iq_DA = np.abs(fft_all)**2
persisted_iq_DA = iq_DA.persist()
persisted_iq_DA

In [ ]:
# fft_DA = xrft.fft(dens_grid_DA, chunks_to_segments=True).mean(['x_segment', 'y_segment', 'z_segment'])
# fft_DA = fft_DA.rename({'freq_y':'qy', 'freq_x':'qx', 'freq_z':'qz'})
# iq_DA = np.abs(fft_DA)**2

In [ ]:
# %%time
with ProgressBar():
    iq_DA = iq_DA.compute()

In [ ]:
iq, qx, qy, qz = convert_grid_qspace(dens_grid, x_axis, y_axis, z_axis)

In [ ]:
plt.close('all')
threshold = 99.9
num_levels = 10
cmap = 'plasma'
fig, ax = plot_3D_grid(iq, qx, qy, qz, cmap, threshold, num_levels)
# fig, ax = plot_3D_grid(iq, qx, qy, qz, cmap, threshold, num_levels)

# ax.set_xlim((-3,3))
# ax.set_ylim((-3,3))
# ax.set_zlim((-3,3))
plt.show()

In [ ]:
del iq_DA

In [ ]:
iq_DA

In [ ]:
persited_iq_DA = iq_DA.persist()

In [ ]:
plt.close('all')
threshold = 99.9
num_levels = 10
cmap = 'plasma'
fig, ax = plot_3D_grid(iq_DA.data.compute(), iq_DA.freq_x.data*2*np.pi, iq_DA.freq_y.data*2*np.pi, iq_DA.freq_z.data*2*np.pi, cmap, threshold, num_levels)
# fig, ax = plot_3D_grid(iq_DA.data, iq_DA.freq_x.data*2*np.pi, iq_DA.freq_y.data*2*np.pi, iq_DA.freq_z.data*2*np.pi, cmap, threshold, num_levels)

plt.show()

# find q-resolutions
### The frequency resolution (qbin size) is given by sampling rate (1/voxel_size) over box size (size of molecule)

In [ ]:
x_vals = qx
y_vals = qy
z_vals = qz
qx_res = x_vals[1]-x_vals[0]
qy_res = y_vals[1]-y_vals[0]
qz_res = z_vals[1]-z_vals[0]
print(f'Resolutions are [qx={qx_res:.4f}, qy={qy_res:.4f}, qz={qz_res:.4f}]')

In [ ]:
x_vals = iq_DA.qx.data*2*np.pi
y_vals = iq_DA.qy.data*2*np.pi
z_vals = iq_DA.qz.data*2*np.pi
qx_res = x_vals[1]-x_vals[0]
qy_res = y_vals[1]-y_vals[0]
qz_res = z_vals[1]-z_vals[0]
print(f'Resolutions are [qx={qx_res:.4f}, qy={qy_res:.4f}, qz={qz_res:.4f}]')

# Set up Detector

In [ ]:
det_pixels = (200,200) #horizontal, vertical
det_qs = (8,8) #horizontal, vertical (these are absolute maximums. detector centered at 0)
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])

psi = 0 #rotation in degrees of detector about detector normal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
phi = 0 #rotation in degrees of detector about detector vertical axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
theta = 0 #rotation in degrees of detector about detector horizontal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)

# plot single detector

In [ ]:
det_ints = intersect_detector(iq, qx, qy, qz, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

# plot
fig, ax1 = subplots()
ax1.imshow(det_ints,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(det_ints, 10), vmax=np.percentile(det_ints, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')

In [ ]:
det_ints = intersect_detector(iq_DA.data, iq_DA.qx.data*2*np.pi, iq_DA.qy.data*2*np.pi, iq_DA.qz.data*2*np.pi, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

# plot
fig, ax1 = subplots()
ax1.imshow(det_ints,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(det_ints, 10), vmax=np.percentile(det_ints, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')

# Generate and sum multiple plots across selected angles

In [ ]:
def generate_detector_ints(det_pixels, det_qs, psi, phi, theta):
    det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
    
    # psi = 0 #rotation in degrees of detector about detector normal axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
    # phi = 0 #rotation in degrees of detector about detector vertical axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
    # theta = 0 #rotation in degrees of detector about detector horizontal axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)
    det_ints = intersect_detector(iq, qx, qy, qz, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

    return det_ints

In [ ]:
#setup detector
det_pixels = (150,150) #horizontal, vertical
det_qs = (6.5,6.5) #horizontal, vertical (these are absolute maximums. detector centered at 0)
psi = 0 #rotation in degrees of detector about detector normal axis
phis = np.linspace(0,180,num=60) #rotation in degrees of detector about detector vertical axis
theta = 0 #rotation in degrees of detector about detector horizontal axis

det_ints = []
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
for i, phi in enumerate(phis):
    det_int = generate_detector_ints(det_pixels, det_qs, psi, phi, theta)
    if i == 0:
        det_sum = det_int
    else:
        det_sum +=det_int
    det_ints.append(det_int)

In [ ]:
%matplotlib widget
fig, ax1 = subplots()
cax = ax1.imshow(det_sum,
           norm=matplotlib.colors.LogNorm(vmin=np.percentile(det_sum, 30), vmax=np.percentile(det_sum, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')
ax1.set_xlim(left=0)
ax1.set_ylim(bottom=0)
cbar = fig.colorbar(cax, ax=ax1)

# Visualize each individual detector across angles

In [ ]:
%matplotlib inline

In [ ]:
for i in range(len(det_ints[:,0,0])):
    det_int = det_ints[i,:,:]
    fig, ax1 = subplots()
    cax = ax1.imshow(det_int,
           norm=matplotlib.colors.LogNorm(vmin=np.percentile(det_int, 10), vmax=np.percentile(det_int, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
    ax1.set_xlabel('q horizontal')
    ax1.set_ylabel('q vertical')
    ax1.set_xlim(0, 3)
    ax1.set_ylim(0, 3)
    cbar = fig.colorbar(cax, ax=ax1)
    ax1.set_title(f'Phi = {i*3} degrees')
    plt.show()
    plt.close('all')
    